In [1]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # labels = y_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator (real classified as ones and generated as zeros)
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (wants discriminator to mistake images as real)
                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % save_interval == 0:
                    self.save_imgs(epoch, global_step)

    def save_imgs(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        generated_images= self.generator.predict(noise)

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_dcgan_cifar10'):
            os.mkdir('images_dcgan_cifar10')
        Image.fromarray(img).save(
            "images_dcgan_cifar10/epoch_%d_step_%d.png" % (epoch, global_step))


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=50, batch_size=64, save_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 1.047152, acc.: 34.38%] [G loss: 0.751971]
epoch:0 step:2 [D loss: 0.853538, acc.: 52.34%] [G loss: 0.838132]
epoch:0 step:3 [D loss: 0.785583, acc.: 51.56%] [G loss: 1.279774]
epoch:0 step:4 [D loss: 0.725562, acc.: 60.94%] [G loss: 1.177330]
epoch:0 step:5 [D loss: 0.774476, acc.: 53.12%] [G loss: 1.292734]
epoch:0 step:6 [D loss: 0.882202, acc.: 51.56%] [G loss: 1.228673]
epoch:0 step:7 [D loss: 0.997823, acc.: 39.84%] [G loss: 1.182061]
epoch:0 step:8 [D loss: 0.792375, acc.: 50.00%] [G loss: 1.196440]
epoch:0 step:9 [D loss: 0.828801, acc.: 50.78%] [G loss: 1.235795]
epoch:0 step:10 [D loss: 0.825721, acc.: 44.53%] [G loss: 1.026727]
epoch:0 step:11 [D loss: 0.743113, acc.: 54.69%] [G loss: 1.034851]
epoch:0 step:12 [D loss: 0.801048, acc.: 49.22%] [G loss: 0.946887]
epoch:0 step:13 [D loss: 0.695298, acc.: 58.59%] [G loss: 1.106979]
epoch:0 step:14 [D loss: 0.816151, acc.: 50.00%] [G loss: 1.231879]
epoch:0 step:15 [D loss: 0.726171, acc.: 59.38%] [G loss:

epoch:0 step:122 [D loss: 0.709930, acc.: 55.47%] [G loss: 1.411904]
epoch:0 step:123 [D loss: 0.804963, acc.: 53.91%] [G loss: 1.428004]
epoch:0 step:124 [D loss: 0.746227, acc.: 56.25%] [G loss: 1.394570]
epoch:0 step:125 [D loss: 0.734188, acc.: 57.81%] [G loss: 1.486595]
epoch:0 step:126 [D loss: 0.825559, acc.: 49.22%] [G loss: 1.219672]
epoch:0 step:127 [D loss: 0.685211, acc.: 59.38%] [G loss: 1.488546]
epoch:0 step:128 [D loss: 0.841445, acc.: 50.00%] [G loss: 1.453935]
epoch:0 step:129 [D loss: 0.761831, acc.: 57.03%] [G loss: 1.306099]
epoch:0 step:130 [D loss: 0.724709, acc.: 56.25%] [G loss: 1.272093]
epoch:0 step:131 [D loss: 0.637404, acc.: 67.19%] [G loss: 1.540234]
epoch:0 step:132 [D loss: 0.686402, acc.: 58.59%] [G loss: 1.349066]
epoch:0 step:133 [D loss: 0.596948, acc.: 67.97%] [G loss: 1.347315]
epoch:0 step:134 [D loss: 0.632998, acc.: 62.50%] [G loss: 1.465476]
epoch:0 step:135 [D loss: 0.535439, acc.: 75.00%] [G loss: 1.459046]
epoch:0 step:136 [D loss: 0.605622

...

epoch:49 step:38566 [D loss: 0.705903, acc.: 49.22%] [G loss: 0.840777]
epoch:49 step:38567 [D loss: 0.691955, acc.: 55.47%] [G loss: 0.826855]
epoch:49 step:38568 [D loss: 0.663248, acc.: 58.59%] [G loss: 0.807525]
epoch:49 step:38569 [D loss: 0.681857, acc.: 54.69%] [G loss: 0.780237]
epoch:49 step:38570 [D loss: 0.624062, acc.: 66.41%] [G loss: 0.779888]
epoch:49 step:38571 [D loss: 0.724307, acc.: 50.78%] [G loss: 0.825236]
epoch:49 step:38572 [D loss: 0.723581, acc.: 46.09%] [G loss: 0.834269]
epoch:49 step:38573 [D loss: 0.764701, acc.: 43.75%] [G loss: 0.708333]
epoch:49 step:38574 [D loss: 0.694611, acc.: 53.12%] [G loss: 0.824105]
epoch:49 step:38575 [D loss: 0.650776, acc.: 60.16%] [G loss: 0.924555]
epoch:49 step:38576 [D loss: 0.757287, acc.: 46.88%] [G loss: 0.822041]
epoch:49 step:38577 [D loss: 0.682811, acc.: 58.59%] [G loss: 1.038777]
epoch:49 step:38578 [D loss: 0.717262, acc.: 50.78%] [G loss: 0.975094]
epoch:49 step:38579 [D loss: 0.740690, acc.: 48.44%] [G loss: 0.

epoch:49 step:38681 [D loss: 0.655854, acc.: 61.72%] [G loss: 0.907419]
epoch:49 step:38682 [D loss: 0.699160, acc.: 46.09%] [G loss: 0.816415]
epoch:49 step:38683 [D loss: 0.735003, acc.: 45.31%] [G loss: 0.780223]
epoch:49 step:38684 [D loss: 0.563554, acc.: 81.25%] [G loss: 0.799398]
epoch:49 step:38685 [D loss: 0.673958, acc.: 53.91%] [G loss: 0.785204]
epoch:49 step:38686 [D loss: 0.667335, acc.: 63.28%] [G loss: 0.877255]
epoch:49 step:38687 [D loss: 0.736660, acc.: 48.44%] [G loss: 0.767491]
epoch:49 step:38688 [D loss: 0.628472, acc.: 68.75%] [G loss: 0.857668]
epoch:49 step:38689 [D loss: 0.681604, acc.: 60.16%] [G loss: 0.906211]
epoch:49 step:38690 [D loss: 0.661852, acc.: 57.03%] [G loss: 0.875063]
epoch:49 step:38691 [D loss: 0.657447, acc.: 64.84%] [G loss: 0.913764]
epoch:49 step:38692 [D loss: 0.673124, acc.: 61.72%] [G loss: 0.882808]
epoch:49 step:38693 [D loss: 0.697377, acc.: 55.47%] [G loss: 0.834075]
epoch:49 step:38694 [D loss: 0.709933, acc.: 45.31%] [G loss: 0.

epoch:49 step:38796 [D loss: 0.666866, acc.: 52.34%] [G loss: 0.797789]
epoch:49 step:38797 [D loss: 0.663794, acc.: 62.50%] [G loss: 0.935454]
epoch:49 step:38798 [D loss: 0.608309, acc.: 68.75%] [G loss: 0.909333]
epoch:49 step:38799 [D loss: 0.620956, acc.: 72.66%] [G loss: 0.881842]
epoch:49 step:38800 [D loss: 0.608516, acc.: 71.09%] [G loss: 0.835436]
epoch:49 step:38801 [D loss: 0.724572, acc.: 50.00%] [G loss: 0.839514]
epoch:49 step:38802 [D loss: 0.632153, acc.: 57.81%] [G loss: 0.715453]
epoch:49 step:38803 [D loss: 0.677055, acc.: 53.12%] [G loss: 0.805682]
epoch:49 step:38804 [D loss: 0.624178, acc.: 67.97%] [G loss: 0.909936]
epoch:49 step:38805 [D loss: 0.700144, acc.: 57.81%] [G loss: 0.920980]
epoch:49 step:38806 [D loss: 0.742781, acc.: 45.31%] [G loss: 0.839568]
epoch:49 step:38807 [D loss: 0.692416, acc.: 53.12%] [G loss: 0.845912]
epoch:49 step:38808 [D loss: 0.731880, acc.: 49.22%] [G loss: 0.779043]
epoch:49 step:38809 [D loss: 0.668593, acc.: 63.28%] [G loss: 0.

epoch:49 step:38911 [D loss: 0.700890, acc.: 49.22%] [G loss: 0.857773]
epoch:49 step:38912 [D loss: 0.723788, acc.: 49.22%] [G loss: 0.756784]
epoch:49 step:38913 [D loss: 0.654469, acc.: 57.81%] [G loss: 0.847656]
epoch:49 step:38914 [D loss: 0.692352, acc.: 59.38%] [G loss: 0.883962]
epoch:49 step:38915 [D loss: 0.608258, acc.: 68.75%] [G loss: 0.770766]
epoch:49 step:38916 [D loss: 0.723552, acc.: 43.75%] [G loss: 0.904429]
epoch:49 step:38917 [D loss: 0.770430, acc.: 42.19%] [G loss: 0.769794]
epoch:49 step:38918 [D loss: 0.677112, acc.: 55.47%] [G loss: 0.928936]
epoch:49 step:38919 [D loss: 0.648981, acc.: 61.72%] [G loss: 0.843043]
epoch:49 step:38920 [D loss: 0.575533, acc.: 67.97%] [G loss: 0.898964]
epoch:49 step:38921 [D loss: 0.634824, acc.: 67.97%] [G loss: 0.837627]
epoch:49 step:38922 [D loss: 0.681458, acc.: 57.81%] [G loss: 0.851950]
epoch:49 step:38923 [D loss: 0.645957, acc.: 61.72%] [G loss: 0.822142]
epoch:49 step:38924 [D loss: 0.677117, acc.: 61.72%] [G loss: 0.

epoch:49 step:39026 [D loss: 0.602781, acc.: 75.78%] [G loss: 0.831044]
epoch:49 step:39027 [D loss: 0.677533, acc.: 57.81%] [G loss: 0.804506]
epoch:49 step:39028 [D loss: 0.738943, acc.: 43.75%] [G loss: 0.802696]
epoch:49 step:39029 [D loss: 0.649680, acc.: 61.72%] [G loss: 0.902496]
epoch:49 step:39030 [D loss: 0.676585, acc.: 53.12%] [G loss: 0.821738]
epoch:49 step:39031 [D loss: 0.651502, acc.: 60.16%] [G loss: 0.876811]
epoch:49 step:39032 [D loss: 0.706199, acc.: 51.56%] [G loss: 0.910137]
epoch:49 step:39033 [D loss: 0.682279, acc.: 57.03%] [G loss: 0.985845]
epoch:49 step:39034 [D loss: 0.638098, acc.: 67.19%] [G loss: 0.889521]
epoch:49 step:39035 [D loss: 0.593122, acc.: 72.66%] [G loss: 0.886180]
epoch:49 step:39036 [D loss: 0.588976, acc.: 73.44%] [G loss: 0.789401]
epoch:49 step:39037 [D loss: 0.750908, acc.: 46.88%] [G loss: 0.786826]
epoch:49 step:39038 [D loss: 0.690800, acc.: 50.78%] [G loss: 0.755396]
epoch:49 step:39039 [D loss: 0.754033, acc.: 42.97%] [G loss: 0.